In [ ]:
from simulate_2 import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

path_eval_data = 'data/batch100'
df_videos = pd.read_csv('data/videos_all_with_result.csv')
df_videos.fillna(0, inplace=True)

df_list = []

for file in os.listdir(path_eval_data):
    
    if file.endswith('.csv'):
        file_path = os.path.join(path_eval_data, file)
        df = pd.read_csv(file_path)
        
        df_list.append(df)

all_data = pd.concat(df_list, ignore_index=True)

models = ['gen2', 'latte', 'pika', 'vgen', 'videocrafter2']
# Bootstrap
G = 100

bs_results = []

for _ in range(G):
    
    bs_df_list = []
    
    #resampling
    for df in df_list:
        indices = np.random.choice(df.index, len(df))
        bs_df_list.append(df.loc[indices])
        
    result = simulate_all(bs_df_list, df_videos)
    
    bs_results.append(result)
    
    print('Bootstrap {} done'.format(_+1))

result_df_list = [pd.DataFrame(columns=models) for _ in range(6)]

for result in bs_results:
    for key in result.keys():
        result_df_list[key-1] = pd.concat([result_df_list[key-1], 
                                           pd.DataFrame([result[key]])], 
                                          ignore_index=True, axis=0)

# mkdir data/bs_result if not exist
if not os.path.exists('data/bs_result'):
    os.makedirs('data/bs_result')

for i in range(6):
    result_df_list[i].to_csv('data/bs_result/bs_result_{}.csv'.format(i+1), index=False)
    